<a href="https://colab.research.google.com/github/AlexanderM-Code/Estructura-Datos/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar las bibliotecas necesarias
!pip install matplotlib networkx scikit-learn ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00


In [2]:
# Importar las bibliotecas necesarias
import random
import numpy as np
from sklearn.cluster import KMeans
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


In [18]:
# Clase que representa una zona en el mapa
class NodoZona:
    def __init__(self, nombre, tipo_zona=None, recursos=None, nivel_dificultad=0):
        """
        Crea una nueva zona con las propiedades indicadas.

        - nombre: el nombre de la zona (ej. "Zona_1").
        - tipo_zona: el tipo de la zona (ej. "Bosque", "Río").
        - recursos: lista de recursos disponibles en la zona (opcional).
        - nivel_dificultad: dificultad de la zona (por defecto 0).
        """
        self.nombre = nombre
        self.tipo_zona = tipo_zona
        self.recursos = recursos if recursos else []
        self.nivel_dificultad = nivel_dificultad

# Clase que genera un mapa procedural
class MapaProcedural:
    def __init__(self, num_zonas=10):
        """
        Crea un nuevo mapa procedural.

        - num_zonas: número total de zonas que tendrá el mapa.
        """
        self.grafo = nx.Graph()  # Grafo para guardar las zonas y conexiones.
        self.num_zonas = num_zonas  # Cantidad de zonas a generar.
        self.tipos_zonas = ["Bosque", "Río", "Montaña", "Llanura", "Desierto", "Costa"]  # Tipos de zonas posibles.
        self.zonas = []  # Lista donde se guardan las zonas creadas.

    def generar_zonas(self):
        """
        Genera las zonas del mapa y les asigna:
        - Un nombre único.
        - Un tipo basado en clusters (usando K-Means).
        - Recursos aleatorios (ej. madera, agua).
        - Nivel de dificultad (entre 1 y 5).
        """
        # Coordenadas aleatorias para las zonas
        datos_zonas = np.array([[random.randint(0, 100), random.randint(0, 100)] for _ in range(self.num_zonas)])

        # Agrupar las zonas según el número de tipos disponibles
        num_clusters = min(len(self.tipos_zonas), self.num_zonas)
        kmeans = KMeans(n_clusters=num_clusters, random_state=0)
        kmeans.fit(datos_zonas)
        etiquetas_zonas = [self.tipos_zonas[label] for label in kmeans.labels_]

        # Crear cada zona con sus propiedades
        for i, tipo in enumerate(etiquetas_zonas):
            nombre_zona = f"Zona_{i}"  # Ejemplo: "Zona_0", "Zona_1"
            nivel_dificultad = random.randint(1, 5)  # Dificultad aleatoria
            recursos = random.sample(["Madera", "Agua", "Piedra", "Comida", "Oro"], k=random.randint(1, 3))  # Recursos aleatorios
            nodo = NodoZona(nombre=nombre_zona, tipo_zona=tipo, recursos=recursos, nivel_dificultad=nivel_dificultad)

            # Agregar la zona al grafo con todos sus datos
            self.grafo.add_node(
                nombre_zona,
                label=f"{nombre_zona} ({tipo})",
                tipo=tipo,
                dificultad=nivel_dificultad,
                recursos=recursos
            )
            self.zonas.append(nodo)

    def conectar_zonas(self):
        """
        Conecta las zonas en el grafo de forma básica:
        - Conexión directa entre zonas consecutivas.
        - Conexiones adicionales aleatorias con cierta probabilidad.
        """
        for i in range(self.num_zonas - 1):
            # Conectar zona actual con la siguiente
            self.grafo.add_edge(f"Zona_{i}", f"Zona_{i + 1}")

            # Conexión aleatoria a otra zona cercana (con 50% de probabilidad)
            if random.random() < 0.5 and i + 2 < self.num_zonas:
                self.grafo.add_edge(f"Zona_{i}", f"Zona_{i + 2}")



In [23]:
def visualizar_mapa(map_obj):
    """
    Dibuja un grafo que representa el mapa procedural con sus zonas y conexiones.

    - Cada tipo de zona tiene un color específico.
    - Las etiquetas muestran detalles como dificultad y recursos disponibles.
    - Se incluye una leyenda para identificar los tipos de zonas.

    map_obj: el objeto del mapa (instancia de MapaProcedural) que contiene las zonas y conexiones.

    """
    colores = {
        "Bosque": "forestgreen",
        "Río": "deepskyblue",
        "Montaña": "saddlebrown",
        "Llanura": "yellowgreen",
        "Desierto": "khaki",
        "Costa": "cornflowerblue"
    }

    nodos_colores = [colores[map_obj.grafo.nodes[nodo]["tipo"]] for nodo in map_obj.grafo.nodes]

    plt.figure(figsize=(14, 12))

    pos = nx.spring_layout(map_obj.grafo, seed=42)

    labels = {
        nodo: f"{data['label']}\nDificultad: {data['dificultad']}\nRecursos: {', '.join(data['recursos'])}"
        for nodo, data in map_obj.grafo.nodes(data=True)
    }

    nx.draw(
        map_obj.grafo, pos, with_labels=False, node_size=2500,
        node_color=nodos_colores, edge_color="gray", linewidths=0.8, font_size=10
    )
    nx.draw_networkx_labels(map_obj.grafo, pos, labels, font_size=8, font_family="sans-serif")

    plt.title("Mapa Procedural de Zonas", fontsize=16)

    plt.gca().set_facecolor('whitesmoke')

    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=zona)
        for zona, color in colores.items()
    ]
    plt.legend(handles=legend_elements, title="Tipos de Zonas", loc="upper right", fontsize=10, title_fontsize=12)

    # Mostrar el gráfico
    plt.show()



In [24]:
def generar_mapa_procedural(num_zonas):
    """
    Genera y visualiza un mapa procedural interactivo.

    - Crea un mapa con la cantidad de zonas especificada.
    - Genera las zonas y las conecta automáticamente.
    - Muestra el grafo del mapa con las zonas y conexiones.

    num_zonas (int): Número de zonas a incluir en el mapa.
    """
    # Crear una instancia de MapaProcedural con el número de zonas indicado
    mapa = MapaProcedural(num_zonas=num_zonas)

    # Generar las zonas y las conexiones del mapa
    mapa.generar_zonas()
    mapa.conectar_zonas()

    # Visualizar el mapa generado
    visualizar_mapa(mapa)

# Crear un control interactivo para ajustar el número de zonas
widgets.interact(
    generar_mapa_procedural,
    num_zonas=widgets.IntSlider(
        value=10, min=5, max=20, step=1, description="Número de Zonas"
    )
)



interactive(children=(IntSlider(value=10, description='Número de Zonas', max=20, min=5), Output()), _dom_class…

<function __main__.generar_mapa_procedural(num_zonas)>